# Realtor Scrapy

Scrapy the real state website and retrieve house listing of a target price and retrieve the info.

### Import Dependencies

In [3]:
import os
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import time
import pprint

# Import DB user and password
from api_keys import positionstack_key

### Setup Splinter (For Mac)

In [2]:
# identify location of chromedriver and store it as a variable
driverPath = !which chromedriver

# Setup configuration variables to enable Splinter to interact with browser
executable_path = {'executable_path': driverPath[0]}
browser = Browser('chrome', **executable_path, headless=False)

### Scraping

In [3]:
# URL of page to be scraped
url_realtor = "https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home/price-"
link_details = "https://www.realtor.com"
min_price = '250000'
max_price = '300000'
sort_by = '/sby-2' # Highest to lowest price
page_number = 1

query_url = f"{url_realtor}{min_price}-{max_price}{sort_by}/pg-{page_number}"
print(query_url)

https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home/price-250000-300000/sby-2/pg-1


### BeautifulSoup

In [4]:
# Scrap with BeautifulSoup. However it does 
# page = requests.get(query_url)
# soup = BeautifulSoup(page.content, 'html.parser')

### Splinter

In [5]:
# Use the browser to visit the url
browser.visit(query_url)

In [6]:
# Wait for 5 seconds for error purpouses
time.sleep(15)

In [7]:
# Return the rendered page by the browser
html_realtor = browser.html

In [8]:
# Use beatifulsoup to scrap the page rendered by the browser
soup = BeautifulSoup(html_realtor, 'html.parser')

In [16]:
# Search for the div where the title is located
results = soup.find_all('div', class_="card-box")
# print(results[1].prettify())
print(f"Total results: {len(results)}")
print('-----------------------------------')

news_title = results[1].text
# print(f"Title: {news_title}")
house_price = results[1].find('span', class_="sc-pZnSc jEfqsK")
# print(f"Price: {house_price.text}")
img_label = results[1].find('img', class_="fade top")
# print(f"Address: {img_label['alt']}")
link_page = results[1].find('a')['href']


print(f"Price: {house_price.text} | Address: {img_label['alt']}")
print(f"Link: {link_details}{link_page}")
print(f"Photo link: {img_label['srcset'].split(',')[1].split(' ')[1]}")

Total results: 48
-----------------------------------
Price: $300,000 | Address: 2620 Starboard Point Dr, Houston, TX 77054
Link: https://www.realtor.com/realestateandhomes-detail/2620-Starboard-Point-Dr_Houston_TX_77054_M80817-72232
Photo link: https://ap.rdcpix.com/49197c51ea457aa035238dbad29a1a23l-m1155526776od-w480_h360_x2.jpg


In [20]:
# Print results and save to a dictionary
n = 0
realstate_list = []
for result in results:
#     Clear the variables to not store repeated info
    house_price = ''
    address = ''
    link_page = ''
    photo_url = ''
                
    n = n + 1
    print('-----------------------------------')
    print('')
    print(f'Result: {n} of {len(results)}')
    if not result.find('div', class_="ads"):
#         print(f'Result: {n} of {len(results)}')
        price_div = result.find('div', class_="price")
        house_price = price_div.find('span').text.split('$')[-1]
        link_page = result.find('a')['href']
        img_label = result.find('img')
        address = img_label['alt']
        print(f"Price: ${house_price} | Address: {address}")
        try:
            price_reduced = result.find('span', class_="price-reduced-amount")
            price_now = price_reduced.text
            print(f"Price reduced: {price_now}")
        except:
            pass
        print(f"Link: {link_details}{link_page}")
        try:
            photo_url = img_label['srcset'].split(',')[1].split(' ')[1]
            print(f"Photo link: {photo_url}")
        except:
            print(f"Photo not available")
    
        # Save results to a dictionary
        realstate_list.append(
            {
                "Price": int(house_price.replace(',','')),
                "Address": address,
                "Link": str(link_details+link_page),
                "Photo link": photo_url
            }
        )

    
    else:
        print('Data not available')

    print('')
    
    

-----------------------------------

Result: 1 of 48
Price: $300,000 | Address: 3506 Chenevert St, Houston, TX 77004
Link: https://www.realtor.com/realestateandhomes-detail/3506-Chenevert-St_Houston_TX_77004_M83831-74404
Photo link: https://ap.rdcpix.com/c32a0999eb72fd3688f358dd5a9631e1l-m4050216900od-w480_h360_x2.jpg

-----------------------------------

Result: 2 of 48
Price: $300,000 | Address: 2620 Starboard Point Dr, Houston, TX 77054
Link: https://www.realtor.com/realestateandhomes-detail/2620-Starboard-Point-Dr_Houston_TX_77054_M80817-72232
Photo link: https://ap.rdcpix.com/49197c51ea457aa035238dbad29a1a23l-m1155526776od-w480_h360_x2.jpg

-----------------------------------

Result: 3 of 48
Price: $300,000 | Address: 3923 Shadow Cove Dr, Houston, TX 77082
Link: https://www.realtor.com/realestateandhomes-detail/3923-Shadow-Cove-Dr_Houston_TX_77082_M74659-12086
Photo link: https://ap.rdcpix.com/8d85bba63c70d3a7ca27d84cfb40875al-m3947304570od-w480_h360_x2.jpg

---------------------

In [ ]:
# When you’ve finished testing, close your browser using browser.quit:
browser.quit()

### Data Cleaning

In [22]:
# Save the data to a dataframe
listing_df = pd.DataFrame(realstate_list)
listing_df.head()

,Price,Address,Link,Photo link
0,300000,"3506 Chenevert St, Houston, TX 77004",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/c32a0999eb72fd3688f358dd...
1,300000,"2620 Starboard Point Dr, Houston, TX 77054",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/49197c51ea457aa035238dba...
2,300000,"3923 Shadow Cove Dr, Houston, TX 77082",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/8d85bba63c70d3a7ca27d84c...
3,300000,"10650 Alcott Dr, Houston, TX 77043",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/8d25df0514bd662282f87167...
4,300000,"650 Westcross St Unit 38, Houston, TX 77018",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/b9545a7302566d196facfde2...


In [23]:
# Dataframe to dictionary
listing_df.to_dict(orient="records")

[{'Price': 300000,
  'Address': '3506 Chenevert St, Houston, TX 77004',
  'Link': 'https://www.realtor.com/realestateandhomes-detail/3506-Chenevert-St_Houston_TX_77004_M83831-74404',
  'Photo link': 'https://ap.rdcpix.com/c32a0999eb72fd3688f358dd5a9631e1l-m4050216900od-w480_h360_x2.jpg'},
 {'Price': 300000,
  'Address': '2620 Starboard Point Dr, Houston, TX 77054',
  'Link': 'https://www.realtor.com/realestateandhomes-detail/2620-Starboard-Point-Dr_Houston_TX_77054_M80817-72232',
  'Photo link': 'https://ap.rdcpix.com/49197c51ea457aa035238dbad29a1a23l-m1155526776od-w480_h360_x2.jpg'},
 {'Price': 300000,
  'Address': '3923 Shadow Cove Dr, Houston, TX 77082',
  'Link': 'https://www.realtor.com/realestateandhomes-detail/3923-Shadow-Cove-Dr_Houston_TX_77082_M74659-12086',
  'Photo link': 'https://ap.rdcpix.com/8d85bba63c70d3a7ca27d84cfb40875al-m3947304570od-w480_h360_x2.jpg'},
 {'Price': 300000,
  'Address': '10650 Alcott Dr, Houston, TX 77043',
  'Link': 'https://www.realtor.com/realestat

### Get Coordinates

In [45]:
# https://nominatim.org/
url_address = 'https://nominatim.openstreetmap.org/search?q='
address_search = '15747+Foxgate+Rd+Houston'
searchformat = '&format=json&polygon_geojson=1&addressdetails=1'
coordinate_search = f"{url_address}{address_search}{searchformat}"
print(coordinate_search)

#  Perform a request for data
response_coordinates = requests.get(coordinate_search).json()

https://nominatim.openstreetmap.org/search?q=15747+Foxgate+Rd+Houston&format=json&polygon_geojson=1&addressdetails=1


In [46]:
response_coordinates

[{'place_id': 291177600,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 15417204,
  'boundingbox': ['29.775993', '29.776093', '-95.635814', '-95.635714'],
  'lat': '29.776043',
  'lon': '-95.635764',
  'display_name': '15747, Foxgate Road, Addicks, Houston, Harris County, Texas, 77079, United States',
  'class': 'place',
  'type': 'house',
  'importance': 0.32100000000000006,
  'address': {'house_number': '15747',
   'road': 'Foxgate Road',
   'hamlet': 'Addicks',
   'city': 'Houston',
   'county': 'Harris County',
   'state': 'Texas',
   'postcode': '77079',
   'country': 'United States',
   'country_code': 'us'}}]

In [55]:
# Format address to search for coordinates
address = listing_df.iloc[0].Address.split(' ')
x = "+".join(address)
address_search = x.replace(',','')
coordinate_search = f"{url_address}{address_search}{searchformat}"
print(address_search)

#  Perform a request for data
response_coordinates = requests.get(coordinate_search).json()
response_coordinates

3506+Chenevert+St+Houston+TX+77004


[{'place_id': 291286915,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 138488478,
  'boundingbox': ['29.735501795918',
   '29.735601795918',
   '-95.371899530612',
   '-95.371799530612'],
  'lat': '29.73555179591837',
  'lon': '-95.37184953061224',
  'display_name': '3506, Chenevert Street, Houston, Harris County, Texas, 77004, United States',
  'class': 'place',
  'type': 'house',
  'importance': 0.5309999999999999,
  'address': {'house_number': '3506',
   'road': 'Chenevert Street',
   'city': 'Houston',
   'county': 'Harris County',
   'state': 'Texas',
   'postcode': '77004',
   'country': 'United States',
   'country_code': 'us'}}]

In [60]:
lat = float(response_coordinates[0]['lat'])
lon = float(response_coordinates[0]['lon'])

In [52]:
for index,row in listing_df.iterrows():
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address.split(' ')
    x = "+".join(address)
    address_search = x.replace(',','')
    coordinate_search = f"{url_address}{address_search}{searchformat}"
    print(address_search)
    
    #  Perform a request for data
    response_coordinates = requests.get(coordinate_search).json()
    print(response_coordinates)

3506+Chenevert+St+Houston+TX+77004
[{'place_id': 291286915, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 138488478, 'boundingbox': ['29.735501795918', '29.735601795918', '-95.371899530612', '-95.371799530612'], 'lat': '29.73555179591837', 'lon': '-95.37184953061224', 'display_name': '3506, Chenevert Street, Houston, Harris County, Texas, 77004, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '3506', 'road': 'Chenevert Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77004', 'country': 'United States', 'country_code': 'us'}}]
2620+Starboard+Point+Dr+Houston+TX+77054
[{'place_id': 291361962, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 251018757, 'boundingbox': ['29.671897', '29.671997', '-95.421159', '-95.421059'], 'lat': '29.671947', 'lon': '-95.4211

[]
10604+Park+Ln+Houston+TX+77093
[{'place_id': 291301532, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15451741, 'boundingbox': ['29.867347244898', '29.867447244898', '-95.347155285714', '-95.347055285714'], 'lat': '29.86739724489796', 'lon': '-95.34710528571428', 'display_name': '10604, Park Lane, Houston, Harris County, Texas, 77093, United States', 'class': 'place', 'type': 'house', 'importance': 0.431, 'address': {'house_number': '10604', 'road': 'Park Lane', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77093', 'country': 'United States', 'country_code': 'us'}}]
10819+Idlebrook+Dr+Houston+TX+77070
[]
3030+Triway+Ln+Houston+TX+77043
[{'place_id': 291393737, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15455919, 'boundingbox': ['29.827468594605', '29.827568594605', '-95.549355503784', '-95.549255503784'], 'lat': '29

[]
209+Cockerel+St+Houston+TX+77018
[{'place_id': 291316164, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15365138, 'boundingbox': ['29.822589333333', '29.822689333333', '-95.400375666667', '-95.400275666667'], 'lat': '29.822639333333335', 'lon': '-95.40032566666666', 'display_name': '209, Cockerel Street, Independence Heights, Houston, Harris County, Texas, 77018, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '209', 'road': 'Cockerel Street', 'neighbourhood': 'Independence Heights', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77018', 'country': 'United States', 'country_code': 'us'}}]
3123+Briarwood+Point+Place+Houston+TX+77063
[]
4712+Libbey+Ln+Houston+TX+77092
[{'place_id': 89839708, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15357936, 'boundingbo

#### Complete Script To Get Coordinates (nominatim)

Note: during the project, it was observed that nominatin API was not able to handle all the addresses.

In [53]:
# https://nominatim.org/
url_address = 'https://nominatim.openstreetmap.org/search?q='
searchformat = '&format=json&polygon_geojson=1&addressdetails=1'
for index,row in listing_df.iterrows():
    print('-------------------------------------------------')
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address.split(' ')
    x = "+".join(address)
    address_search = x.replace(',','')
    coordinate_search = f"{url_address}{address_search}{searchformat}"
    print(address_search)
    
    #  Perform a request for data
    response_coordinates = requests.get(coordinate_search).json()
    print(response_coordinates)

-------------------------------------------------
3506+Chenevert+St+Houston+TX+77004
[{'place_id': 291286915, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 138488478, 'boundingbox': ['29.735501795918', '29.735601795918', '-95.371899530612', '-95.371799530612'], 'lat': '29.73555179591837', 'lon': '-95.37184953061224', 'display_name': '3506, Chenevert Street, Houston, Harris County, Texas, 77004, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '3506', 'road': 'Chenevert Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77004', 'country': 'United States', 'country_code': 'us'}}]
-------------------------------------------------
2620+Starboard+Point+Dr+Houston+TX+77054
[{'place_id': 291361962, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 251018757, 'boundi

[{'place_id': 291220665, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15416739, 'boundingbox': ['29.841783444444', '29.841883444444', '-95.405213888889', '-95.405113888889'], 'lat': '29.841833444444443', 'lon': '-95.4051638888889', 'display_name': '410, Hohldale Street, Independence Heights, Houston, Harris County, Texas, 77091, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '410', 'road': 'Hohldale Street', 'neighbourhood': 'Independence Heights', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77091', 'country': 'United States', 'country_code': 'us'}}]
-------------------------------------------------
13414+Boca+Raton+Dr+Houston+TX+77069
[]
-------------------------------------------------
10604+Park+Ln+Houston+TX+77093
[{'place_id': 291301532, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyri

[{'place_id': 291375960, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15431605, 'boundingbox': ['29.829487963215', '29.829587963215', '-95.473068959065', '-95.472968959065'], 'lat': '29.829537963215355', 'lon': '-95.47301895906455', 'display_name': '5503, Cheshire Lane, Oak Forest, Rosslyn, Houston, Harris County, Texas, 77092, United States', 'class': 'place', 'type': 'house', 'importance': 0.431, 'address': {'house_number': '5503', 'road': 'Cheshire Lane', 'neighbourhood': 'Oak Forest', 'hamlet': 'Rosslyn', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77092', 'country': 'United States', 'country_code': 'us'}}]
-------------------------------------------------
26+N+Champion+Villa+Dr+Houston+TX+77069
[]
-------------------------------------------------
1812+Iris+Knoll+Ln+Pearland+TX+77089
[]
-------------------------------------------------
2649+Bellwick+Canyon+Ct+Pearland+TX+77089
[]
-

In [79]:
def get_coordinates(address):    
    # https://nominatim.org/
    url_address = 'https://nominatim.openstreetmap.org/search?q='
    searchformat = '&format=json&polygon_geojson=1&addressdetails=1'
    
    # Format address to search for coordinates
    address_string = address.Address.split(' ')
    x = "+".join(address_string)
    address_search = x.replace(',','')
    coordinate_search = f"{url_address}{address_search}{searchformat}"
    
    #  Perform a request for data
    response_coordinates = requests.get(coordinate_search).json()
    
    # return coordinates
    lat = round(float(response_coordinates[0]['lat'])*100)/100
    lon = round(float(response_coordinates[0]['lon'])*100)/100
    
    map_url = f"https://www.openstreetmap.org/#map=15/{lat}/{lon}"
    print(f"Map: {map_url}")

    return {"latitude": lat, "longitude": lon, "map": map_url}

In [80]:
address_cord = get_coordinates(listing_df.iloc[index])

In [81]:
address_cord['latitude']
# address_cord['longitude']

29.77

In [85]:
coordinates = get_coordinates(listing_df.iloc[index])
coordinates['latitude']

In [86]:
coordinates['latitude']

29.77

#### Complete Script To Get Coordinates (positionstack)

https://positionstack.com/quickstart

In [32]:
def positiontrack_coordinates(address):
    # Forward Geocoding API Endpoint
    url_positiontrack = f"http://api.positionstack.com/v1/forward?access_key={positionstack_key}&query="
    
    address_lookup = address.replace(" ", "%20")
    url_coordinates = f"{url_positiontrack}{address_lookup}& limit=1"

    print(f"Query URL:")
    print(url_coordinates)
    print('')

    #  Perform a request for data
    response_coordinates = requests.get(url_coordinates).json()
    print(f"Results:")
    print(response_coordinates)
    print('')

    lat = response_coordinates['data'][0]['latitude']
    lon = response_coordinates['data'][0]['longitude']

    print(f"Latitude: {lat}")
    print(f"Longitude: {lon}")
    
    map_url = f"https://www.openstreetmap.org/?mlat={lat}&mlon={lon}#map=15/{lat}/{lon}"
    
    print(f"Map: {map_url}")

    return {"latitude": lat, "longitude": lon, "map": map_url}
    
address = "1600 Pennsylvania Ave NW, Washington DC"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=1600%20Pennsylvania%20Ave%20NW,%20Washington%20DC& limit=1

Results:
{'data': [{'latitude': 38.897675, 'longitude': -77.036547, 'type': 'address', 'name': '1600 Pennsylvania Avenue NW', 'number': '1600', 'postal_code': '20500', 'street': 'Pennsylvania Avenue NW', 'confidence': 1, 'region': 'District of Columbia', 'region_code': 'DC', 'county': None, 'locality': 'Washington', 'administrative_area': None, 'neighbourhood': 'White House Grounds', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '1600 Pennsylvania Avenue NW, Washington, DC, USA'}]}

Latitude: 38.897675
Longitude: -77.036547
Map: https://www.openstreetmap.org/?mlat=38.897675&mlon=-77.036547#map=15/38.897675/-77.036547


{'latitude': 38.897675,
 'longitude': -77.036547,
 'map': 'https://www.openstreetmap.org/?mlat=38.897675&mlon=-77.036547#map=15/38.897675/-77.036547'}

In [24]:
# Example of query for coordinates
address = "650 Westcross St Unit 38, Houston, TX 77018"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=650%20Westcross%20St%20Unit%2038,%20Houston,%20TX%2077018

Results:
{'data': [{'latitude': 29.829312, 'longitude': -95.409167, 'type': 'address', 'name': '650 Westcross St', 'number': '650', 'postal_code': '77018', 'street': 'Westcross St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Independence Heights', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '650 Westcross St, TX, USA'}]}

Latitude: 29.829312
Longitude: -95.409167


In [29]:
# Example of query for coordinates
address = "17511 Lonesome Dove Trl, Houston, TX 77095"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=17511%20Lonesome%20Dove%20Trl,%20Houston,%20TX%2077095

Results:
{'data': [{'latitude': 29.934773, 'longitude': -95.678385, 'type': 'address', 'name': '17511 Lonesome Dove Trl', 'number': '17511', 'postal_code': '77095', 'street': 'Lonesome Dove Trl', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': None, 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '17511 Lonesome Dove Trl, TX, USA'}]}

Latitude: 29.934773
Longitude: -95.678385
https://www.openstreetmap.org/#map=15/29.934773/-95.678385


{'latitude': 29.934773, 'longitude': -95.678385}

In [28]:
# Example of query for coordinates
address = "10418 Springland Ct, Houston,TX 77065"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=10418%20Springland%20Ct,%20Houston,TX%2077065

Results:
{'data': [{'latitude': 29.924385, 'longitude': -95.591604, 'type': 'address', 'name': '10418 Springland Ct', 'number': '10418', 'postal_code': '77065', 'street': 'Springland Ct', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Willowbrook', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '10418 Springland Ct, TX, USA'}]}

Latitude: 29.924385
Longitude: -95.591604
https://www.openstreetmap.org/#map=15/29.924385/-95.591604


{'latitude': 29.924385, 'longitude': -95.591604}